In [21]:
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

measurement = np.load('../dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs']

In [22]:
# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6  # Channel 6 is EVE (illegitimate)

# Extract data for ALICE and BOB channels
alice_CIRs = data_cir[:100, alice_channel, :, :]  # Shape: (9797, 251, 2)
eve_CIRs = data_cir[:100, eve_channel, :, :]  # Shape: (9797, 251, 2)

# ALICE features - real, imaginary, magnitude
alice_real = alice_CIRs[:, :, 0]
alice_imag = alice_CIRs[:, :, 1]
alice_magnitude = np.abs(alice_real + 1j * alice_imag)
alice_features = np.hstack((alice_real, alice_imag, alice_magnitude))


# EVE features - real, imaginary, magnitude
eve_real = eve_CIRs[:, :, 0]
eve_imag = eve_CIRs[:, :, 1]
eve_magnitude = np.abs(eve_real + 1j * eve_imag)
eve_features = np.hstack((eve_real, eve_imag, eve_magnitude))

# Atoms for our dictionary - Reshape Alice's data to flatten each sample.
alice_atoms = alice_features.reshape(100, -1)  # Each sample becomes a long vector.
eve_atoms = eve_features.reshape(100, -1)

# Create labels for Alice and Eve.
alice_labels = np.zeros(alice_atoms.shape[0])  # Label '0' for Alice.
eve_labels = np.ones(eve_atoms.shape[0])       # Label '1' for Eve.

# Combine data and labels.
atoms = np.vstack((alice_atoms, eve_atoms))  # (200, 753)
true_labels = np.hstack((alice_labels, eve_labels))  # (200,)


In [23]:
# Step 3: Split the Data into Training and Test Sets
trainData, testData, trainLabel, testLabel = train_test_split(atoms, true_labels, test_size=0.2, random_state=42, stratify=true_labels)

# Step 4: Form the Dictionary D
D = trainData.T

# trainData (160, 753)
# testData (40, 753)
# trainLabel (160,)
# testLabel (40,)

In [24]:
# Step 5: Sparse Coding Function
def find_sparse_coefficients(tSample, D, n_nonzero_coefs=10):
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)
    omp.fit(D, tSample)
    return omp.coef_

# D - (753, 160)
# tSample - (753,)

In [25]:
# Step 6: Function to Calculate Residuals for Each Class
def calculate_residual(tSample, D, coefficients, class_indices):
    coef_class = np.zeros_like(coefficients)
    coef_class[class_indices] = coefficients[class_indices]  # Keep onltSample coefficients for the specified class
    reconstructed_signal = D @ coef_class
    residual = np.linalg.norm(tSample - reconstructed_signal)
    # print(f'Dictionary: {D.shape}')
    # print(f'Coeff class: {coef_class.shape}')
    # print(f'Reconstructed signal{reconstructed_signal.shape}')
    return residual

# tSample - (753,)
# coefficients - (160,)
# class_indices - (80,)
# reconstructed_signal - (753,)


In [26]:
# Step 7: Classification Function
def classify_signal(tSample, D, trainLabel):
    
    # tSample - (753,)
    # D - (753, 160)
    # trainLabel - (160,)
    # coefficients - (160,)
    
    # Find sparse coefficients for the new signal
    coefficients = find_sparse_coefficients(tSample, D)
    # print(coefficients)
    
    # Initialize residuals list
    residuals = []

    # Calculate residual for each class
    unique_classes = np.unique(trainLabel) # 0 and 1
    for class_label in unique_classes:
        class_indices = np.where(trainLabel == class_label)[0]  # Indices of columns in D belonging to this class
        residual = calculate_residual(tSample, D, coefficients, class_indices)
        residuals.append(residual)

    # Predict the class with the smallest residual
    predicted_class = unique_classes[np.argmin(residuals)]
    return predicted_class


In [27]:
# Step 8: Classifying Test Data and Evaluating the Model

# testData (40, 753)
predictions = []
for testSample in testData:
    predicted_class = classify_signal(testSample, D, trainLabel)
    predictions.append(predicted_class)


# Convert predictions to a numpy array for comparison
predictions = np.array(predictions)
# print(predictions)

# Step 9: Calculate Accuracy
accuracy = np.mean(predictions == testLabel)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

Classification Accuracy: 85.00%


In [28]:
# Calculate confusion matrix
print(f"\nTotal testing channel: {testData.shape[0]*2}")

# print(true_labels.shape)
# print(predictions.shape)
tn, fp, fn, tp = confusion_matrix(testLabel, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# # Missed Detection Rate (MDR)
MDR = fp / (fp + tn)

# # False Alarm Rate (FAR)
FAR = fn / (fn + tp)

# # Gamma calculation
gamma = (tp + fn) / (tn + fp)

# # Authentication Rate (AR)
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp))

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")


Total testing channel: 80
tp: 20
tn: 14
fp: 6
fn: 0
MDR: 0.3
FAR: 0.0
AR: 0.85
